In [40]:
# pip install librosa scikit-learn
# !pip install keras
# !pip install tensorflow
# !pip install scikeras
# !pip install resampy

                                              0.0/3.1 MB ? eta -:--:--
     ------                                   0.5/3.1 MB 15.9 MB/s eta 0:00:01
     ----------------                         1.3/3.1 MB 16.0 MB/s eta 0:00:01
     ----------------------------             2.2/3.1 MB 17.3 MB/s eta 0:00:01
     ---------------------------------------  3.1/3.1 MB 17.8 MB/s eta 0:00:01
     ---------------------------------------  3.1/3.1 MB 17.8 MB/s eta 0:00:01
     ---------------------------------------- 3.1/3.1 MB 12.3 MB/s eta 0:00:00


In [2]:
#MODELO 1

import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os


# Carregar os dados de áudio e extrair características
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# Diretório onde os arquivos de áudio estão armazenados
audio_directory = "audios classificados/"

# Lista de arquivos de áudio e seus rótulos
audio_files = []
labels = []

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):  # Certifique-se de que são arquivos de áudio
        # Extrair o rótulo do nome do arquivo
        label = 1 if "positivo" in filename.lower() else 0

        # Adicionar o arquivo e o rótulo às listas
        audio_files.append(os.path.join(audio_directory, filename))
        labels.append(label)

# Extrair características de cada áudio
X = np.array([extract_features(file) for file in audio_files])
y = np.array(labels)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir e treinar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

Acurácia do modelo: 63.64%


In [5]:
#MODELO 2

import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Diretório onde os arquivos de áudio estão armazenados
audio_directory = "audios classificados/"

# Lista de arquivos de áudio e seus rótulos
audio_files = []
labels = []

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        label = 1 if "positivo" in filename.lower() else 0
        audio_files.append(os.path.join(audio_directory, filename))
        labels.append(label)

# Extrair características de áudio
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# Extrair características de todos os áudios
X = np.array([extract_features(file) for file in audio_files])
y = np.array(labels)

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)#, random_state=42)

# Construir modelo de rede neural
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Camada de saída com ativação sigmoid para retornar um valor entre 0 e 1

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Avaliar o modelo
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.7722 - accuracy: 0.5517 - val_loss: 0.8094 - val_accuracy: 0.3750
Epoch 2/10
1/1 [==============================] - 0s 45ms/step - loss: 0.6493 - accuracy: 0.6552 - val_loss: 0.7936 - val_accuracy: 0.3750
Epoch 3/10
1/1 [==============================] - 0s 47ms/step - loss: 0.5576 - accuracy: 0.8276 - val_loss: 0.7800 - val_accuracy: 0.6250
Epoch 4/10
1/1 [==============================] - 0s 49ms/step - loss: 0.4927 - accuracy: 0.8276 - val_loss: 0.7699 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 46ms/step - loss: 0.4432 - accuracy: 0.9310 - val_loss: 0.7628 - val_accuracy: 0.3750
Epoch 6/10
1/1 [==============================] - 0s 51ms/step - loss: 0.4056 - accuracy: 0.9655 - val_loss: 0.7569 - val_accuracy: 0.3750
Epoch 7/10
1/1 [==============================] - 0s 52ms/step - loss: 0.3741 - accuracy: 0.9655 - val_loss: 0.7520 - val_accuracy: 0.3750
Epoch 8/10
1/1 [=============

In [32]:
#BUSCA DE HIPERPARÂMETROS

from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier 
import tensorflow as tf

# Função que cria o modelo
def create_model(optimizer='adam', loss='binary_crossentropy',
                 batch_size=32, epochs=10, verbose=0):
    model = Sequential()
    model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Utiliza a classe Adam do módulo tf.keras.optimizers.legacy
    optimizer = tf.keras.optimizers.legacy.Adam() if optimizer == 'adam' else optimizer
    
    metrics = ['accuracy'] if loss != 'hinge' else ['accuracy', 'precision']
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

# Crie o modelo KerasClassifier para ser usado no GridSearchCV
model = KerasClassifier(model=create_model, verbose=0)

# Defina os parâmetros a serem testados
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30],
    'optimizer': ['adam', 'rmsprop'],
    'loss': ['binary_crossentropy', 'hinge']
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)

# Execute a busca
grid_result = grid.fit(X_train, y_train)

# Imprima os melhores resultados
print("Melhor Acurácia:", grid_result.best_score_)
print("Melhores Parâmetros:", grid_result.best_params_)


C:\Users\Victor\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
54 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Victor\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Victor\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\Victor\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "C:\Users\Victor\anaconda3\Lib\site

Melhor Acurácia: 0.7072649572649573
Melhores Parâmetros: {'batch_size': 64, 'epochs': 10, 'loss': 'binary_crossentropy', 'optimizer': 'adam'}


In [53]:
#MELHOR MODELO

import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Diretório onde os arquivos de áudio estão armazenados
audio_directory = "audios classificados/"

# Lista de arquivos de áudio e seus rótulos
audio_files = []
labels = []

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        label = 1 if "positivo" in filename.lower() else 0
        audio_files.append(os.path.join(audio_directory, filename))
        labels.append(label)

# Extrair características de áudio
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# Extrair características de todos os áudios
X = np.array([extract_features(file) for file in audio_files])
y = np.array(labels)

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)#, random_state=42)


# Defina os melhores parâmetros encontrados na busca
best_params = {'batch_size': 64, 'epochs': 10, 'loss': 'binary_crossentropy', 'optimizer': 'adam'}

# Crie e compile o modelo com os melhores parâmetros
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam() if best_params['optimizer'] == 'adam' else best_params['optimizer']
model.compile(optimizer=optimizer, loss=best_params['loss'], metrics=['accuracy'])

# Treine o modelo
model.fit(X_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=1)

# Avalie o modelo nos dados de teste
accuracy = model.evaluate(X_test, y_test)[1]
print("Acurácia do modelo nos dados de teste:", accuracy)

# Salve o modelo em um arquivo .h5
model.save("seu_modelo.h5")


Epoch 1/10
1/1 [==============================] - 1s 681ms/step - loss: 0.7146 - accuracy: 0.3514
Epoch 2/10
1/1 [==============================] - 0s 7ms/step - loss: 0.6204 - accuracy: 0.6486
Epoch 3/10
1/1 [==============================] - 0s 7ms/step - loss: 0.5485 - accuracy: 0.7838
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.4920 - accuracy: 0.8919
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 0.4452 - accuracy: 0.9189
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.4056 - accuracy: 0.9459
Epoch 7/10
1/1 [==============================] - 0s 7ms/step - loss: 0.3718 - accuracy: 0.9459
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 0.3429 - accuracy: 0.9459
Epoch 9/10
1/1 [==============================] - 0s 7ms/step - loss: 0.3171 - accuracy: 0.9189
Epoch 10/10
1/1 [==============================] - 0s 114ms/step - loss: 0.8406 - accuracy: 0.4118
Acurácia do modelo nos dados de tes

C:\Users\Victor\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [54]:
import os
import librosa
import numpy as np
from keras.models import load_model

# Função para extrair características do áudio
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# Diretório contendo os arquivos de áudio
audio_directory = 'audios classificados/'

# Carregar o modelo treinado
model = load_model("seu_modelo.h5")

# Loop pelos arquivos no diretório
for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        # Construir o caminho completo do arquivo
        file_path = os.path.join(audio_directory, filename)

        # Extrair características do áudio
        audio_features = extract_features(file_path)

        # Ajustar as características para corresponder à entrada do modelo
        audio_features_input = np.array(audio_features).reshape(1, -1)

        # Obter o score de sentimento usando o modelo treinado
        sentiment_score = model.predict(audio_features_input)

        # Exibir o resultado para cada arquivo
        print(f"Arquivo: {filename}, Score de Sentimento: {sentiment_score[0][0]}")


1/1 [==============================] - 0s 55ms/step
Arquivo: negativo - 1.wav, Score de Sentimento: 0.0020549313630908728
1/1 [==============================] - 0s 22ms/step
Arquivo: negativo - 10.wav, Score de Sentimento: 0.000646142871119082
1/1 [==============================] - 0s 20ms/step
Arquivo: negativo - 11.wav, Score de Sentimento: 0.005195104517042637
1/1 [==============================] - 0s 25ms/step
Arquivo: negativo - 12.wav, Score de Sentimento: 0.9985334873199463
1/1 [==============================] - 0s 27ms/step
Arquivo: negativo - 13.wav, Score de Sentimento: 1.9160440842824755e-06
1/1 [==============================] - 0s 19ms/step
Arquivo: negativo - 14.wav, Score de Sentimento: 0.9999302625656128
1/1 [==============================] - 0s 24ms/step
Arquivo: negativo - 15.wav, Score de Sentimento: 0.9999697804450989
1/1 [==============================] - 0s 26ms/step
Arquivo: negativo - 16.wav, Score de Sentimento: 0.0035558121744543314
1/1 [======================